<a href="https://colab.research.google.com/github/ask-1710/FIRE2021-OffensiveLanguageDetection/blob/main/Tamil_OffensiveLanguageDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import f1_score
import numpy as np
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
tokenizer_train  = Tokenizer(num_words = 100, oov_token="<OOV>")
data_train = pd.read_excel('/content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/train/Tamil-Codemixed_offensive_data_Training-Tweet-HL.xlsx', names=["Name","Text","result"], index_col=False)
corpus_train=[]
data_train['Text'] = data_train['Text'].apply(lambda x: x.lower())
for i in data_train['Text'].tolist():
  if(str(i)!='nan'):
    corpus_train.append(i)

label_train=[]
for i in data_train['result'].tolist():
  if(str(i)!='nan'):
    label_train.append(i)


tokenizer_train .fit_on_texts(corpus_train)
total_words_train  = len(tokenizer_train .word_index) + 1

print('Mapping of Words and numbers: '+str(tokenizer_train .word_index))
print('Total number of Words for training: '+str(total_words_train ))


tokenizer = tokenizer_train

data = pd.read_csv('/content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/train/Tamil_hasoc_tanglish_test_withlabels.tsv',sep='\t',header=0, names=["Name","Text","result"], index_col=False)
corpus=[]
data['Text'] = data['Text'].astype('str')
data['Text'] = data['Text'].apply(lambda x:x.lower())

for i in data['Text'].tolist():
  if(str(i)!='nan'):
    corpus.append(i)

label=[]
for i in data['result'].tolist():
  if(str(i)!='nan'):
    label.append(i)


Mapping of Words and numbers: {'<OOV>': 1, 'user': 2, 'unaku': 3, 'tag': 4, 'la': 5, 'da': 6, 'romba': 7, 'nu': 8, 'ah': 9, 'punda': 10, 'rt': 11, 'oru': 12, 'enna': 13, 'ku': 14, 'nee': 15, 'than': 16, 'unga': 17, 'na': 18, 'illa': 19, 'koothi': 20, 'manasu': 21, 'thevidiya': 22, 'iruku': 23, 'un': 24, 'intha': 25, 'ne': 26, 'poi': 27, 'en': 28, 'antha': 29, 'bro': 30, 'panna': 31, 'and': 32, 'nalla': 33, 'iruka': 34, 'avan': 35, 'vera': 36, 'dei': 37, 'athu': 38, 'ena': 39, 'panni': 40, 'vijay': 41, 'tha': 42, 'a': 43, 'indha': 44, 'anna': 45, 'but': 46, 'thaan': 47, 'tweet': 48, 'va': 49, 'pundai': 50, 'enga': 51, 'loosu': 52, 'neenga': 53, 'ya': 54, 'ellam': 55, 'the': 56, 'enaku': 57, 'pola': 58, 'mattum': 59, 'sir': 60, 'kuda': 61, 'lam': 62, 'yen': 63, 'fans': 64, 'potta': 65, 'to': 66, 'pannu': 67, 'dhan': 68, 'i': 69, 'vanthu': 70, 'kooda': 71, 'mathiri': 72, 'ada': 73, 'naan': 74, 'ipdi': 75, 'ipo': 76, 'ajith': 77, '\xa0': 78, 'poda': 79, 'so': 80, 'you': 81, 'amma': 82, 'oo

In [5]:
input_sequences = []
labels = []
for line in range(len(corpus)):
  if label[line] == 'OFF':
    l = 0
  else:
    l = 1
  token_list = tokenizer.texts_to_sequences([corpus[line]])[0]
  input_sequences.append(token_list)
  labels.append(l)

# pad sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=100, padding='pre'))

# create predictors and label
xs = input_sequences[:,:]
ys = tf.keras.utils.to_categorical(labels, num_classes=2)

print('Number of Validation samples:', end = ' ')
print(len(ys),len(xs))

#TRAIN
labels = []
input_sequences = []
for line in range(len(corpus_train)):
  if label_train[line] == 'OFF':
    l = 0
  else:
    l = 1

  token_list_train = tokenizer.texts_to_sequences([corpus_train[line]])[0]
  input_sequences.append(token_list_train)
  labels.append(l)
  
# pad sequences 
input_sequences = np.array(pad_sequences(input_sequences, maxlen=100, padding='pre'))

print("Number of training samples: ",end='')
xs_train= input_sequences[:,:]
ys_train = tf.keras.utils.to_categorical(labels, num_classes=2)
print(len(ys_train),len(xs_train))

Number of Validation samples: 939 939
Number of training samples: 3999 3999


In [6]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint 

In [7]:
model = Sequential()
model.add(Embedding(total_words_train, 100, input_length=100))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(2, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

In [8]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

In [9]:
ModelCheckpointcheckpoint = ModelCheckpoint('/content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/train/best_model.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

In [10]:
history = model.fit(xs_train, ys_train, epochs=10, verbose=1,validation_data=(xs, ys), callbacks=[earlystop, ModelCheckpointcheckpoint])

Epoch 1/10
125/125 [==============================] - 47s 350ms/step - loss: 0.5387 - accuracy: 0.7049 - val_loss: 0.4562 - val_accuracy: 0.7508

Epoch 00001: val_loss improved from inf to 0.45616, saving model to /content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/train/best_model.h5
Epoch 2/10
125/125 [==============================] - 32s 252ms/step - loss: 0.4519 - accuracy: 0.7734 - val_loss: 0.3858 - val_accuracy: 0.7721

Epoch 00002: val_loss improved from 0.45616 to 0.38580, saving model to /content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/train/best_model.h5
Epoch 3/10
125/125 [==============================] - 32s 252ms/step - loss: 0.4068 - accuracy: 0.7969 - val_loss: 0.4432 - val_accuracy: 0.7082

Epoch 00003: val_loss did not improve from 0.38580
Epoch 4/10
125/125 [==============================] - 32s 253ms/step - loss: 0.3884 - accuracy: 0.8057 - val_loss: 0.4078 - val_accuracy: 0.7465

Epoch 00004: val_loss did not improve from 0.38580


In [21]:
# Compute f1-score 
y_pred = model.predict(xs).argmax(axis=-1)
y_true = ys.argmax(axis=-1)
print(f1_score(y_true, y_pred))

0.6979695431472082


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 100)          1693900   
_________________________________________________________________
bidirectional (Bidirectional (None, 300)               301200    
_________________________________________________________________
dense (Dense)                (None, 2)                 602       
Total params: 1,995,702
Trainable params: 1,995,702
Non-trainable params: 0
_________________________________________________________________


In [ ]:
!pip install emoji
import emoji

     |████████████████████████████████| 185 kB 5.1 MB/s 
  Created wheel for emoji: filename=emoji-1.5.0-py3-none-any.whl size=187457 sha256=7115f7356d4a3e2da55c7c45115368b0e88a6167f86a4c860c09016fe67e112d
  Stored in directory: /root/.cache/pip/wheels/db/b5/f6/b39abf14e94b3d6640613bbe630a66c10ccf7a12882d064fb5
Successfully built emoji


In [ ]:
import re

In [ ]:
categories = ['Off','Not']

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/test/hasoc_tamil_task2_withoutlabels.tsv', sep='\t',names=["id","text","label"], index_col=False)

In [ ]:
test_df['text'] = test_df['text'].apply(lambda x: x.lower())

Demotify

In [ ]:
def deEmojify(text):
  emoji_pattern = re.compile("["
                  u"\U0001F600-\U0001F64F"  # emoticons
                  u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                  u"\U0001F680-\U0001F6FF"  # transport & map symbols
                  u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                  "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'',text)

In [ ]:
# def give_emoji_free_text(text):
#     allchars = [str for str in text.decode('utf-8')]
#     emoji_list = [c for c in allchars if c in emoji.UNICODE_EMOJI]
#     clean_text = ' '.join([str for str in text.decode('utf-8').split() if not any(i in str for i in emoji_list)])
#     return clean_text
# test_df['text'] = test_df['text'].apply(lambda x:give_emoji_free_text(x.encode('utf8')))

In [ ]:
# !pip install indic-nlp-library
# from indicnlp.transliterate.unicode_transliterate import UnicodeIndicTransliterator
# from indicnlp.transliterate.unicode_transliterate import ItransTransliterator

In [ ]:
test_df['text'] = test_df['text'].apply(lambda x:deEmojify(x))

In [ ]:
!pip install langdetect
from langdetect import detect

     |████████████████████████████████| 981 kB 5.1 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=c0443f242c21778bbd6a25ef8f0c095fd24d7344442d621b877343fe8caf0610
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [ ]:
# !pip install open-tamil

In [ ]:
!pip install indic-transliteration
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

     |████████████████████████████████| 119 kB 5.2 MB/s 
     |████████████████████████████████| 662 kB 28.6 MB/s 
     |████████████████████████████████| 904 kB 43.5 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20


In [ ]:
# transliterate('ஆழகே ஆழகே ஏதுவும்', sanscript.TAMIL, sanscript.ITRANS)

'Azhaghe Azhaghe edhuvuM'

In [ ]:
def testing(test_sentence):
  try:
    if test_sentence and detect(test_sentence) == "ta":
      test_sentence = transliterate(test_sentence, sanscript.TAMIL, sanscript.ITRANS)
      test_sentence = test_sentence.lower()
      print('TAMIL SCRIPT TRANSILERATED: ', test_sentence)
      
    tokens = tokenizer.texts_to_sequences(test_sentence)[0]
    if len(tokens)>0:
      ans = model.predict(tokens)
      return categories[np.argmax(ans[0])]
    else:
      print('Exception : Unseen words found')
      return 'Not'
  except Exception as e:
    print(e)
    return 'Not'

In [ ]:
test_df['label'] = test_df['text'].apply(lambda x: testing(x))

Exception : Unseen words found
Exception : Unseen words found
Exception : Unseen words found
Exception : Unseen words found
Exception : Unseen words found
Exception : Unseen words found
Exception : Unseen words found
Exception : Unseen words found
Exception : Unseen words found
Exception : Unseen words found
Exception : Unseen words found
Exception : Unseen words found
Exception : Unseen words found
TAMIL SCRIPT TRANSILERATED:  ni~ngha ghadhaijhiya ghedhdha ghelvighghu bhadhil mudhalil dhirumana dhaghaval maiyadhdhil jadhi bhèyarai nighgha vendhum,bhallighal marrum ghallurighalilum jadhi jhaனridhazh nighghabhbhadha vendhum
TAMIL SCRIPT TRANSILERATED:  indha 2 jhambhava~nghal madhdhum alla, iனru aனaidhdhu jhèyalghalilum valiyavar èliyavarai vadhaidhdhu ghòndhu dhaன irughghirar. odhdhughghagha, vari bhanadhdhirghagha madhdhum vidhdhu vaidhdhirughghirarghal. indha 2 jhambhava~nghal nalidhazhil vèlivandhu maraighghabhbhadhdha vijhayam dhaன. ghandhum ghanadha maனidharghal dhaன i~nghu adhigh

In [ ]:
len(test_df[test_df['label']=='Not']) / len(test_df)

0.25874125874125875

In [ ]:
len(test_df[test_df['label']=='Off']) / len(test_df)

0.7412587412587412

In [ ]:
test_df.to_csv('/content/drive/MyDrive/Datasets/HASOC-Dravidian-CodeMix-FIRE 2021/test/tamil-results.csv', index=False)